This is a project in "Python Intermediate" course of Dataquest. In this project, we'll work with a data set of submissions to popular technology site Hacker News.

Hacker News is a site started by the startup incubator Y Combinator, where user-submitted stories (known as "posts") are voted and commented upon, similar to reddit. Hacker News is extremely popular in technology and startup circles, and posts that make it to the top of Hacker News' listings can get hundreds of thousands of visitors as a result.

The full data set is [here](https://www.kaggle.com/hacker-news/hacker-news-posts), in this project, it has been reduced from almost 300,000 rows to approximately 20,000 rows by removing all submissions that did not receive any comments, and then randomly sampling from the remaining submissions. Below are descriptions of the columns:

- id: The unique identifier from Hacker News for the post
- title: The title of the post
- url: The URL that the posts links to, if it the post has a URL
- num_points: The number of points the post acquired, calculated as the total number of upvotes minus the total number of downvotes
- num_comments: The number of comments that were made on the post
- author: The username of the person who submitted the post
- created_at: The date and time at which the post was submitted

Here are the first few rows of the data set:

<table>
 <thead>
  <tr style='height:43.5pt'>
   <td>id
   </td>
   <td>title
   </td>
   <td>url
   </td>
   <td>num_points
   </td>
   <td>num_comments
   </td>
   <td>author
   </td>
   <td>created_at
   </td>
  </tr>
 </thead>
 <tr>
  <td>12224879</td>
  <td>Interactive Dynamic Video</td>
  <td>http://www.interactivedynamicvideo.com/</td>
  <td>386</td>
  <td>52</td>
  <td>ne0phyte</td>
  <td>8/4/2016 11:52</td>
 </tr>
  <tr>
  <td>10975351</td>
  <td>How to Use Open Source and Shut the F*ck Up at the Same Time</td>
  <td>http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/</td>
  <td>39</td>
  <td>10</td>
  <td>josep2</td>
  <td>1/26/2016 19:30</td>
 </tr>
</table>


## Data Loading

In [ ]:
from csv import reader
import datetime as dt

In [ ]:
def read_csv_file(filename):
    opened_file = open(filename)
    file_content = reader(opened_file)
    lst = list(file_content)
    opened_file.close()
    
    return lst[0], lst[1:]

def display_table(dataset, start, end, information=False):
    ds = dataset[start:end]
    for row in ds:
        print(row)
        print('\n') # adds a new (empty) line after each row
        
    if (information == True):
        print('Number of rows:', len(ds))
        print('Number of columns:', len(ds[0]))            

In [ ]:
headers, hn = read_csv_file("/kaggle/input/hacker-news-posts/HN_posts_year_to_Sep_26_2016.csv")
print(headers)
print("\n")
display_table(hn, 0, 5, True)

In [ ]:
ask_posts = []
show_posts = []
other_posts = []

for row in hn:
    title = row[1]
    title = title.lower()
    if (title.startswith("ask hn")):
        ask_posts.append(row)
    elif (title.startswith("show hn")):
        show_posts.append(row)
    else:
        other_posts.append(row)
        
print(len(ask_posts))
print(len(show_posts))
print(len(other_posts))

In [ ]:
ask_posts[:5]

In [ ]:
show_posts[:5]

In [ ]:
def avg_comments(posts):
    total_comments = 0
    
    for row in posts:
        num_comments = int(row[4])
        total_comments += num_comments

    avg_comments = total_comments/len(posts)

    return avg_comments

In [ ]:
avg_ask_comments = avg_comments(ask_posts)
avg_show_comments = avg_comments(show_posts)
print("The average comment of ask posts is {:.2f}".format(avg_ask_comments))
print("The average comment of show posts is {:.2f}".format(avg_show_comments))

Since ask posts are more likely to receive comments, we'll focus our remaining analysis just on these posts.

Next, we'll determine if ask posts created at a certain time are more likely to attract comments. We'll use the following steps to perform this analysis:

- Calculate the amount of ask posts created in each hour of the day, along with the number of comments received.
- Calculate the average number of comments ask posts receive by hour created.

In [ ]:
result_list = []
for row in ask_posts:
    created_dt_str = row[6] #created_at column
    num_comments = int(row[4]) #num_comments column
    
    result_list.append([created_dt_str, num_comments])

In [ ]:
result_list[:5]

In [ ]:
counts_by_hour = {}
comments_by_hour = {}

for row in result_list:
    created_dt_str = row[0]
    num_comments = int(row[1]) #num_comments column
    
    # convert to datetime type
    created_dt = dt.datetime.strptime(created_dt_str, "%m/%d/%Y %H:%M")
    # extract the hours of the date
    hour = created_dt.hour
    
    if hour in comments_by_hour:
        comments_by_hour[hour] += num_comments
        counts_by_hour[hour] += 1
    else:
        comments_by_hour[hour] = num_comments
        counts_by_hour[hour] = 1      

So far, we created two dictionaries:

- counts_by_hour: contains the number of ask posts created during each hour of the day.

- comments_by_hour: contains the corresponding number of comments ask posts created at each hour received.

In [ ]:
counts_by_hour

In [ ]:
comments_by_hour

Next, we'll use these two dictionaries to calculate the average number of comments for posts created during each hour of the day.

In [ ]:
avg_by_hour = []
for hour in counts_by_hour:
    cnt = counts_by_hour[hour]
    num_comments = comments_by_hour[hour]
    
    avg_by_hour.append([hour, round(num_comments/cnt, 2)])

In [ ]:
avg_by_hour

Let's finish by sorting the list of lists and printing the five highest values in a format that's easier to read.

In [ ]:
swap_avg_by_hour = []
for row in avg_by_hour:
    swap_avg_by_hour.append([row[1], row[0]])

In [ ]:
swap_avg_by_hour

In [ ]:
sorted_swap = sorted(swap_avg_by_hour, reverse=True)

In [ ]:
sorted_swap

Print the string "Top 5 Hours for Ask Posts Comments".

In [ ]:
for row in sorted_swap[:5]:
    print("{}:00: {} average comments per post".format(row[1], row[0]))

Based on the result, we can see that the appropriate time for creating a post to have a higher chance of receiving comments is around 15:00 to 15:59. 